In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from random import seed, randrange

## Purpose

The purpose of this notebook is to process the load data published as part of the Australian Government's *Smart City, Smart Grid* programme run in the city of Newcastle in NSW.

For the purposes of performing power flow analysis on a test network we need to create an average daily profile. As the *Smart Grid Smart City* programme aimed to alter the consumption behaviour of trial participants by offering them various products, we will only use the load profiles of households in the control group. 

The descriptions of net and gross generation are unclear for this dataset. One would assume that gross generation would always be greater than net, however this is untrue at times for this data. Consequently, it is difficult to derive the total consumption of a household with solar installed. Thus, households in the control group will also be excluded from this analysis.   

The original data can be found [here](http://datagovau.s3-ap-southeast-2.amazonaws.com/CDINTERVALREADINGALLNOQUOTES.csv.7z)

The customer metadata can be found [here](https://data.gov.au/dataset/ds-dga-4e21dea3-9b87-4610-94c7-15a8a77907ef/distribution/dist-dga-0404c872-8a83-40e6-9c04-88dfec125aee/details?q=)

The dataset column descriptions can be found [here](https://data.gov.au/dataset/ds-dga-4e21dea3-9b87-4610-94c7-15a8a77907ef/distribution/dist-dga-52e630d3-725c-4854-81bf-5d3a4f3c0385/details?q=)


In [2]:
data_file_name = "CD_INTERVAL_READING_ALL_NO_QUOTES.csv"
metadata_file_name = "sgsc-ct_customer-household-data-revised.csv"
data_dir = Path("./in")
data_path = data_dir / data_file_name
metadata_path = data_dir / metadata_file_name

out_path = Path("./out")
out_path.mkdir(exist_ok=True, parents=True)

## Find all the customer IDs of customers in the control group

In [3]:
metadata_df = pd.read_csv(metadata_path.resolve())
# We only want those customers in the control group
metadata_df = metadata_df[metadata_df["CONTROL_GROUP_FLAG"] == "Y"]
customers_with_solar = metadata_df[metadata_df["GROSS_SOLAR_CNT"] > 0]
print(
    f"Number of customers in control group: {len(metadata_df['CUSTOMER_KEY'].unique())}"
)
print(
    f"Number of customers in control group with solar: {len(customers_with_solar['CUSTOMER_KEY'].unique())}"
)

Number of customers in control group: 2093
Number of customers in control group with solar: 25


## Looks like there's not too many customers with solar in the control group

In [4]:
control_customer_ids = metadata_df[metadata_df["GROSS_SOLAR_CNT"] == 0]["CUSTOMER_KEY"]
print(f"Number of customers in control group: {len(control_customer_ids.unique())}")

Number of customers in control group: 2068


## Filter the full dataset, retaining only those customers in the control group without solar

In [ ]:
# Check if the processed data already exists
control_out_path = (out_path / "control_group.csv").resolve()
if not control_out_path.exists():
        print("Couldn't find processed control group data, creating")
        control_df = pd.DataFrame()
        # Read dataset in chunks as it's too large for memory
        chunks = pd.read_csv(data_path.resolve(), parse_dates=["READING_DATETIME"], chunksize=100000)
        # Append rows of original dataset associated with customers in the control group to the control df
        # This will take a while, you may want to tune the chunksize for your RAM size. 
        # Or load it all into memory if you have enough (you'll need more than 60GB though)
        for chunk_df in chunks:
            control_rows = chunk_df[chunk_df["CUSTOMER_ID"].isin(control_customer_ids)]
            control_df = control_df.append(control_rows)
        control_df.to_csv(control_out_path)
else:
    print("Found processed control group data, reading from disk")
    control_df = pd.read_csv(control_out_path)

## Process the filtered data into a single profile

In [ ]:
control_df = control_df if not control_df.empty else pd.read_csv((out_path / "control_group.csv").resolve())


## Final data processing
Now that we've finished processing the data we will take a brief look at it and create an average profile.  

In [ ]:
in_path = out_path / "aggregated.csv"
df = pd.read_csv(in_path.resolve())

In [ ]:
# Let's take a look at the number of unique customers
unique_sites = df["CUSTOMER_ID"].unique()
print(f"Unique sites: {len(unique_sites)}")
seed(42)

In [ ]:
net_gen_pos = df["net_generation_kwh"] > 0
gross_gen_pos = df["gross_generation_kwh"] > 0
# df[df["net_generation_kwh"] > df["gross_generation_kwh"]]
df[df["net_generation_kwh"] < 0]

In [ ]:
column_mapper = {col: col.strip(" ").lower() for col in list(df.columns)}
df = df.rename(columns=column_mapper)

In [ ]:
random_site_customer_id = unique_sites[randrange(0, len(unique_sites))]
random_site = df[df["customer_id"] == random_site_customer_id]
random_site.plot(x="time", y="gross_generation_kwh")